In [3]:
!pip install -q accelerate peft bitsandbytes transformers trl

## Fine-Tuning LLaMA 2

In [13]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from transformers import BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer # Supervised fine-tuning
from huggingface_hub import notebook_login

In [5]:
# Set environment variables for memory efficiency
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# Enable memory-efficient attention mechanisms
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)

In [6]:
# Model configuration
base_model = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "llama-2-7b-chat-guanaco"
compute_dtype = getattr(torch, "float16")

# Quantization configuration
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,  # Enable double quantization for memory efficiency
)

# Load the model with quantization settings
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load and configure tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Load dataset
dataset = load_dataset(dataset_name)
if "train" in dataset:
    train_dataset = dataset["train"]
else:
    train_dataset = dataset  # If dataset is not split


In [8]:
# LoRA configuration with reduced parameters for memory efficiency
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=16,  # rank
    bias="none",
    task_type="CAUSAL_LM",
)

In [9]:
# Training arguments with reduced batch size and increased gradient accumulation
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,  # Reduced batch size for memory efficiency
    gradient_accumulation_steps=4,  # Increased to compensate for smaller batch size
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,  # Enable mixed precision for memory efficiency
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to=['none']
)

trainer = SFTTrainer(
    model=model,
    args=training_params,
    train_dataset=train_dataset,
    peft_config=peft_params,
)

# Start training
print("Starting training...")
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training...


Step,Training Loss
25,1.345400
50,1.618800
75,1.210800
100,1.430700
125,1.175700
150,1.353900
175,1.169900
200,1.450600
225,1.153700
250,1.517300


TrainOutput(global_step=250, training_loss=1.3426793212890624, metrics={'train_runtime': 1367.7666, 'train_samples_per_second': 0.731, 'train_steps_per_second': 0.183, 'total_flos': 1.673178221039616e+16, 'train_loss': 1.3426793212890624})

In [10]:
# Save the model
print(f"Saving model to {new_model}...")
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)
print("Training complete and model saved!")

Saving model to llama-2-7b-chat-guanaco...
Training complete and model saved!


In [14]:
# login to HF
notebook_login()

In [ ]:
# Define the repository name for Hugging Face
output_dir = f'{base_model.split("/")[-1]}-{new_model}'
repo_name = f'agoor97/{output_dir}'

print(f"Pushing model to Hugging Face Hub: {repo_name}")

# Push model and tokenizer to the Hub
trainer.model.push_to_hub(repo_name, commit_message="Training Complete")
tokenizer.push_to_hub(repo_name, commit_message="Training Complete")

print("Model successfully pushed to Hugging Face Hub!")

----